In [35]:
import sys
sys.path.insert(0, '/VIS_3TH/')

import numpy as np
from numba import njit

from setup import SHORT_PATH
import importlib.util
game_name = 'Durak'

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2

In [36]:
@njit()
def player(state, per):
    validActions = getValidActions(state)
    validActions = np.where(validActions == 1)[0]

    spades = validActions[(validActions>=0) & (validActions<12)]
    clubs = validActions[(validActions>=13) & (validActions<25)]
    diamonds = validActions[(validActions>=26) & (validActions<38)]
    hearts = validActions[(validActions>=39) & (validActions<51)]

    master = np.where(state[158:162] == 1)[0][0]

    if (state[157] == 1) and (np.sum(state[104:156]) != 0) :
        card = np.where(state[104:156] == 1)[0][0]
        if ((card // 13 == master) or (card % 13 >= 7)) and (state[162]>5):
            return 52, per

    if master == 0:
        if (len(clubs) > 0):
            return clubs[0], per
        if (len(diamonds) > 0):
            return  diamonds[0], per
        if (len(hearts) > 0):
            return hearts[0], per
        if len(spades)>0:
            return spades[0], per
    elif master == 1:
        if (len(diamonds) > 0):
            return  diamonds[0], per
        if (len(hearts) > 0):
            return hearts[0], per
        if len(spades)>0:
            return spades[0], per
        if (len(clubs) > 0):
            return clubs[0], per
    elif master == 2:
        if (len(hearts) > 0):
            return hearts[0], per
        if len(spades)>0:
            return spades[0], per
        if (len(clubs) > 0):
            return clubs[0], per
        if (len(diamonds) > 0):
            return  diamonds[0], per
    elif master == 3:
        if len(spades)>0:
            return spades[0], per
        if (len(clubs) > 0):
            return clubs[0], per
        if (len(diamonds) > 0):
            return  diamonds[0], per
        if (len(hearts) > 0):
            return hearts[0], per
    
    action = validActions[np.random.randint(len(validActions))]
    return action, per

In [37]:
win, per = numba_main_2(player, 1000, np.array([]), 1)
win

334